In [1]:
from operator import itemgetter

import opencc
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain_community.callbacks.manager import get_openai_callback
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai.chat_models.azure import AzureChatOpenAI
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_together.llms import Together

load_dotenv()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

system_prompt = """
You are a science communicator specializing in astronomy. Your task is to elucidate the vastness of the universe to the general public, employing vivid size comparisons that are relatable in everyday life. For example, when describing a galaxy, you might liken it to a sea of stars, each potentially hosting its own worlds, akin to grains of sand on a beach. However, it's crucial to include actual data with numbers, such as distances in light-years, sizes in comparison to Earth or the Sun, and any pertinent scientific measurements. Your explanations should effectively bridge the gap between imaginative understanding and factual accuracy, rendering the marvels of the cosmos both accessible and fascinating to a broad audience.
"""

user_prompt = """
{question}
"""


def s2hk(content):
    converter = opencc.OpenCC("s2hk")
    return converter.convert(content)


def get_answer(question, system_prompt, user_prompt, **kwargs):
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template(user_prompt),
        ]
    )

    # Use kwargs to override default parameters if provided
    model_params = {
        "model": "gpt-3.5-turbo",
        "temperature": 0.7,
        "max_tokens": 4096,
        "max_retries": 1000,  # Overload the rate limit per minute
    }
    model_params.update(kwargs)

    model = ChatOpenAI(**model_params)
    # model = AzureChatOpenAI(**model_params)
    # model = Together(**model_params)

    # model = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.7, max_tokens=4096)
    # model = AzureChatOpenAI(model="gpt-4-turbo-preview", temperature=0.7, max_tokens=4096)
    # model = Together(model="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.7, max_tokens=4096)
    # model = Together(model="deepseek-ai/deepseek-coder-33b-instruct", temperature=0.7, max_tokens=4096)

    chain = (
        RunnablePassthrough.assign(
            chat_history=RunnableLambda(
                memory.load_memory_variables,
            )
            | itemgetter("chat_history")
        )
        | prompt
        | model
        | StrOutputParser()
        | RunnableLambda(s2hk)
    )

    with get_openai_callback() as callback:
        response = chain.invoke({"question": question})
        print(callback, end="\n\n")
    print(response)
    memory.save_context({"question": question}, {"response": response})


# memory.load_memory_variables({})

In [2]:
get_answer("Describe Sagittarius A* and TON 618.", system_prompt, user_prompt, n=2)

Tokens Used: 881
	Prompt Tokens: 161
	Completion Tokens: 720
Successful Requests: 1
Total Cost (USD): $0.0016814999999999998

Sagittarius A* and TON 618 are both fascinating objects in the cosmos that showcase the immense scales and powers at play in our universe.

Let's start with Sagittarius A*. This enigmatic object is a supermassive black hole located at the center of our Milky Way galaxy, approximately 26,000 light-years away from Earth. To put this into perspective, if we imagine Earth as a tiny speck, Sagittarius A* would be like a colossal cosmic vacuum cleaner, with a mass equivalent to about 4 million times that of our Sun, all packed into a region smaller than our solar system. The gravitational pull of Sagittarius A* is so strong that not even light can escape its grasp, hence the name "black hole."

Moving on to TON 618, we are now delving into the realm of supermassive black holes on a mind-boggling scale. TON 618 is one of the most massive black holes known to astronomer